In [8]:
# JetBot Object Detection and Navigation
# This notebook detects bottles and cases, navigates toward them, and takes photos upon reaching them

import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import cv2
import numpy as np
import time
import os
from datetime import datetime
import ipywidgets
from IPython.display import display
from jetbot import Robot, Camera

# Create output directory for photos
OUTPUT_DIR = 'captured_photos'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

# Define the target classes to detect (from COCO dataset)
# COCO class indices: bottle=39, suitcase=28
TARGET_CLASSES = {
    'bottle': 39,
    'case': 28,
    'phone': 67,
    'book': 73,
    'cup': 41,
    'keyboard': 66,
    'mouse': 64,
    'backpack': 24,
    'chair': 56,
    'apple': 47,
    'remote': 65,
    'laptop': 63
}

# Initialize the JetBot
robot = Robot()

# Initialize the camera
camera = Camera.instance(width=224, height=224)

# Function to load the pre-trained object detection model (SSD MobileNet)
def load_model():
    # Load pre-trained SSD MobileNet model from torchvision
    model = torchvision.models.detection.ssd300_vgg16(pretrained=True)
    model.eval().cuda()
    return model

# Function to preprocess the image for the model
def preprocess(image):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return transform(image).unsqueeze(0).cuda()

# Function to detect objects in the image
def detect_objects(model, image, confidence_threshold=0.5):
    # Convert image to RGB format
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Preprocess the image
    input_tensor = preprocess(image_rgb)
    
    # Perform inference
    with torch.no_grad():
        predictions = model(input_tensor)
    
    # Extract detection results
    boxes = predictions[0]['boxes'].cpu().numpy()
    labels = predictions[0]['labels'].cpu().numpy()
    scores = predictions[0]['scores'].cpu().numpy()
    
    # Filter detections based on confidence threshold
    mask = scores > confidence_threshold
    boxes = boxes[mask]
    labels = labels[mask]
    scores = scores[mask]
    
    return boxes, labels, scores

# Function to find target objects in the detections
def find_target_objects(labels, boxes, scores):
    target_indices = {}
    target_boxes = {}
    target_scores = {}
    
    for target_name, target_class in TARGET_CLASSES.items():
        # Find indices where the label matches the target class
        indices = np.where(labels == target_class)[0]
        if len(indices) > 0:
            # Get the index of the detection with the highest confidence score
            best_idx = indices[np.argmax(scores[indices])]
            target_indices[target_name] = best_idx
            target_boxes[target_name] = boxes[best_idx]
            target_scores[target_name] = scores[best_idx]
    
    return target_indices, target_boxes, target_scores

# Function to calculate the center of a bounding box
def get_box_center(box):
    x1, y1, x2, y2 = box
    return (x1 + x2) / 2, (y1 + y2) / 2

# Function to calculate the area of a bounding box
def get_box_area(box):
    x1, y1, x2, y2 = box
    return (x2 - x1) * (y2 - y1)

# Function to draw bounding boxes on an image
def draw_boxes(image, boxes, labels, scores):
    # Create a copy of the image
    image_with_boxes = image.copy()
    
    # Define colors for each class (in BGR)
    colors = {
        TARGET_CLASSES['bottle']: (0, 255, 0),  # Green for bottle
        TARGET_CLASSES['case']: (0, 0, 255)     # Red for case
    }
    
    # Draw each bounding box
    for box, label, score in zip(boxes, labels, scores):
        x1, y1, x2, y2 = box.astype(int)
        color = colors.get(label, (255, 0, 0))  # Default blue for other objects
        
        # Draw rectangle
        cv2.rectangle(image_with_boxes, (x1, y1), (x2, y2), color, 2)
        
        # Add label and score
        label_text = f"Class: {label}, {score:.2f}"
        for target_name, target_class in TARGET_CLASSES.items():
            if label == target_class:
                label_text = f"{target_name}: {score:.2f}"
                break
                
        cv2.putText(image_with_boxes, label_text, (x1, y1 - 10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    return image_with_boxes

# Function to navigate towards a target
def navigate_to_target(robot, center_x, image_width, target_area, full_image_area, stop_threshold=0.4):
    # Calculate the relative position of the target in the frame
    image_center_x = image_width / 2
    offset = center_x - image_center_x
    
    # Calculate the normalized offset (-1 to 1)
    normalized_offset = offset / image_center_x
    
    # Calculate area ratio to determine distance
    area_ratio = target_area / full_image_area
    
    # Determine if we're close enough to stop
    if area_ratio > stop_threshold:
        # We've reached the target, stop the robot
        robot.stop()
        return True
    
    # Set motor speeds based on the target position
    # The further the target is from the center, the more we turn
    # The larger the target appears, the slower we move
    
    # Base speed decreases as we get closer to the target
    base_speed = max(0.3, 0.7 - area_ratio)
    
    # Calculate left and right motor speeds
    left_speed = base_speed
    right_speed = base_speed
    
    # Adjust speeds based on the target's horizontal position
    if normalized_offset < -0.1:  # Target is to the left
        # Turn left by decreasing right motor speed
        right_speed += min(0.4, abs(normalized_offset) * 0.8)
        left_speed -= min(0.3, abs(normalized_offset) * 0.4)
    elif normalized_offset > 0.1:  # Target is to the right
        # Turn right by decreasing left motor speed
        left_speed += min(0.4, normalized_offset * 0.8)
        right_speed -= min(0.3, normalized_offset * 0.4)
    
    # Ensure speeds are within valid range
    left_speed = max(0.0, min(1.0, left_speed))
    right_speed = max(0.0, min(1.0, right_speed))
    
    # Apply motor speeds
    robot.left_motor.value = left_speed
    robot.right_motor.value = right_speed
    
    return False

# Function to take a photo and save it
def take_photo(image, object_name):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{OUTPUT_DIR}/{object_name}_{timestamp}.jpg"
    cv2.imwrite(filename, image)
    print(f"Photo saved as {filename}")

# Create display widgets
image_widget = ipywidgets.Image(format='jpeg', width=500, height=500)
display(image_widget)

# Button to start and stop the robot
run_button = ipywidgets.ToggleButton(description='Start / Stop')
display(run_button)

# Load the object detection model
print("Loading model...")
model = load_model()
print("Model loaded!")

# Main execution loop
def execute_detection_and_navigation():
    running = False
    target_reached = {target: False for target in TARGET_CLASSES}
    
    try:
        while True:
            # Check if the button state has changed
            if run_button.value != running:
                running = run_button.value
                if not running:
                    robot.stop()
                    print("Stopped")
            
            # Capture image from camera
            image = camera.value
            
            if image is not None:
                # Convert the image to BGR format for OpenCV processing
                image_bgr = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
                height, width = image_bgr.shape[:2]
                full_image_area = height * width
                
                # Only perform detection if the robot is running
                if running:
                    # Detect objects in the image
                    boxes, labels, scores = detect_objects(model, image_bgr)
                    
                    # Find target objects
                    target_indices, target_boxes, target_scores = find_target_objects(labels, boxes, scores)
                    
                    # Check if any target object is found
                    if target_boxes:
                        # Select the target with the highest confidence score
                        selected_target = max(target_scores.items(), key=lambda x: x[1])[0]
                        selected_box = target_boxes[selected_target]
                        
                        # Calculate the center of the selected target
                        center_x, _ = get_box_center(selected_box)
                        
                        # Calculate the area of the target
                        target_area = get_box_area(selected_box)
                        
                        # Navigate towards the target
                        reached = navigate_to_target(robot, center_x, width, target_area, full_image_area)
                        
                        # If we've reached the target and haven't taken a photo yet
                        if reached and not target_reached[selected_target]:
                            take_photo(image_bgr, selected_target)
                            target_reached[selected_target] = True
                            print(f"Reached {selected_target}!")
                            # Wait a moment to avoid taking multiple photos
                            time.sleep(2)
                        elif not reached:
                            # Reset the target_reached status if we move away from the target
                            target_reached[selected_target] = False
                    else:
                        # No target found, stop the robot
                        robot.stop()
                
                # Always draw boxes for better visualization
                if 'boxes' in locals() and len(boxes) > 0:
                    image_with_boxes = draw_boxes(image_bgr, boxes, labels, scores)
                else:
                    image_with_boxes = image_bgr
                
                # Convert back to RGB for display
                image_rgb = cv2.cvtColor(image_with_boxes, cv2.COLOR_BGR2RGB)
                
                # Update the image widget
                image_widget.value = cv2.imencode('.jpg', image_rgb)[1].tobytes()
            
            # Brief pause to prevent high CPU usage
            time.sleep(0.05)
            
    except KeyboardInterrupt:
        robot.stop()
        print("Program stopped")
    except Exception as e:
        robot.stop()
        print(f"Error: {e}")
        raise

# Run the main function in a separate cell using:
# execute_detection_and_navigation()
print("Run the function execute_detection_and_navigation() to start the navigation")

# Example usage:
# execute_detection_and_navigation()

Image(value=b'', format='jpeg', height='500', width='500')

ToggleButton(value=False, description='Start / Stop')

Loading model...
Model loaded!
Run the function execute_detection_and_navigation() to start the navigation


In [9]:
execute_detection_and_navigation()

Program stopped


In [7]:
camera.stop()

In [7]:
import time
import numpy as np
from jetbot import Robot
from jetbot import Camera
from PIL import Image as PILImage
import torch
import torch.nn as nn
from torchvision.models import mobilenet_v2
import torchvision.transforms as transforms
import os
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
import cv2

# --- CONFIGURABLE PARAMETERS ---
PRIORITY_ORDER = ["bottle", "case"]  # Define your own order here
OBJECT_IMAGES_PATH = "dataset/"  # Folder containing bottle.jpg, case.jpg, bag.jpg
SAVE_PATH = "logs/"              # Folder to save visit pictures and logs
DETECTION_THRESHOLD = 0.6        # Minimum similarity threshold for object detection
SCAN_SPEED = 0.3               # Speed for scanning rotation
APPROACH_SPEED = 0.15            # Speed for approaching objects
SCAN_STEP_TIME = 0.3             # Time between scan steps
MAX_APPROACH_TIME = 5.0          # Maximum time to approach an object (seconds)
CENTER_THRESHOLD = 30            # Pixels from center to consider object centered

# Ensure save directory exists
os.makedirs(SAVE_PATH, exist_ok=True)

# --- SETUP ---
robot = Robot()
camera = Camera.instance(width=224, height=224)

# Create widgets for visualization
image_widget = widgets.Image(format='jpeg', width=224, height=224)
status_widget = widgets.Text(value='Initializing...', description='Status:')
display(image_widget, status_widget)

# Define the function to convert BGR to JPEG for display
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])

# Link camera to image widget
camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

# Load MobileNetV2 for feature extraction
model = mobilenet_v2(pretrained=True)
# Remove the classifier to get features instead of classification
feature_extractor = nn.Sequential(*list(model.children())[:-1])
feature_extractor.eval()

# Image preprocessing pipeline
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Object name to feature vector dictionary
object_features = {}

# Load object reference images and extract features
def load_reference_images():
    status_widget.value = "Loading reference images..."
    for name in PRIORITY_ORDER:
        img_path = os.path.join(OBJECT_IMAGES_PATH, f"{name}.jpg")
        try:
            image = PILImage.open(img_path).convert('RGB')
            image_tensor = transform(image).unsqueeze(0)
            
            # Extract features
            with torch.no_grad():
                features = feature_extractor(image_tensor)
                features = features.view(features.size(0), -1)  # Flatten
                object_features[name] = features
                
            print(f"Loaded reference image for {name}")
        except Exception as e:
            print(f"Error loading {name}: {e}")
    
    status_widget.value = f"Loaded {len(object_features)} reference objects"

# Cosine similarity for feature comparison
def cosine_similarity(t1, t2):
    t1_norm = torch.nn.functional.normalize(t1, p=2, dim=1)
    t2_norm = torch.nn.functional.normalize(t2, p=2, dim=1)
    return torch.mm(t1_norm, t2_norm.transpose(0, 1)).item()

# Capture current frame and compare with reference objects
def identify_object():
    frame = camera.value
    if frame is None:
        return None, 0.0
        
    # Convert BGR to RGB (OpenCV uses BGR, PIL expects RGB)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = PILImage.fromarray(frame_rgb)
    img_tensor = transform(img).unsqueeze(0)
    
    # Extract features
    with torch.no_grad():
        features = feature_extractor(img_tensor)
        features = features.view(features.size(0), -1)  # Flatten
    
    # Calculate similarity with each reference object
    similarities = {}
    for name, ref_features in object_features.items():
        sim = cosine_similarity(features, ref_features)
        similarities[name] = sim
    
    # Get the most similar object above threshold
    best_match = max(similarities.items(), key=lambda x: x[1])
    if best_match[1] > DETECTION_THRESHOLD:
        return best_match
    else:
        return None, 0.0

# Detect object in frame and get bounding box (placeholder using template matching)
def detect_object_position(frame, object_name):
    h, w = frame.shape[:2]
    center_x, center_y = w // 2, h // 2
    
    # This is a simplified detection method using template matching
    # In a real scenario, you might want to use a proper object detection model
    try:
        # Load the reference image
        ref_path = os.path.join(OBJECT_IMAGES_PATH, f"{object_name}.jpg")
        template = cv2.imread(ref_path)
        template = cv2.resize(template, (w // 4, h // 4))  # Resize template
        
        # Convert both images to grayscale
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
        
        # Perform template matching
        result = cv2.matchTemplate(frame_gray, template_gray, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        
        # Calculate center of matched region
        th, tw = template_gray.shape
        top_left = max_loc
        obj_center_x = top_left[0] + tw // 2
        obj_center_y = top_left[1] + th // 2
        
        # Draw rectangle on visualization
        vis_frame = frame.copy()
        cv2.rectangle(vis_frame, top_left, (top_left[0] + tw, top_left[1] + th), (0, 255, 0), 2)
        cv2.circle(vis_frame, (obj_center_x, obj_center_y), 5, (0, 0, 255), -1)
        cv2.putText(vis_frame, f"{object_name}: {max_val:.2f}", 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # Update display
        image_widget.value = bgr8_to_jpeg(vis_frame)
        
        # Calculate offset from center (normalized to -1 to 1)
        x_offset = (obj_center_x - center_x) / (w / 2)
        y_offset = (obj_center_y - center_y) / (h / 2)
        
        return (x_offset, y_offset, max_val)
    except Exception as e:
        print(f"Error in position detection: {e}")
        return (0, 0, 0)

# Scan environment for objects
def scan_objects():
    status_widget.value = "Scanning for objects..."
    detected = set()
    detected_with_conf = {}
    
    # Do a full 360° scan in smaller steps
    for _ in range(24):  # 15° steps for full rotation
        robot.left(SCAN_SPEED)
        time.sleep(SCAN_STEP_TIME)
        robot.stop()
        time.sleep(0.2)  # Stabilize camera
        
        # Identify object
        obj, confidence = identify_object()
        if obj:
            detected.add(obj)
            detected_with_conf[obj] = max(confidence, detected_with_conf.get(obj, 0))
            status_widget.value = f"Found {obj} (conf: {confidence:.2f})"
            
        # Display progress
        found_str = ", ".join([f"{o}:{c:.2f}" for o, c in detected_with_conf.items()])
        print(f"Scan progress: {found_str}")
        
        # If all objects found, break early
        if all(obj in detected for obj in PRIORITY_ORDER):
            break
    
    robot.stop()
    status_widget.value = f"Scan complete. Found: {', '.join(detected)}"
    return detected

# Approach an object
def go_to_object(target):
    status_widget.value = f"Moving to {target}..."
    start_time = time.time()
    
    # Try to approach the object
    while time.time() - start_time < MAX_APPROACH_TIME:
        frame = camera.value
        if frame is None:
            continue
            
        # Check if we're seeing the target object
        obj, confidence = identify_object()
        
        if obj == target and confidence > DETECTION_THRESHOLD:
            # Get position and calculate steering adjustment
            x_offset, y_offset, match_conf = detect_object_position(frame, target)
            
            # If we're very close (object fills much of the frame or is centered)
            if abs(x_offset) < (CENTER_THRESHOLD / 112):  # 112 is half of 224
                # Go straight if centered
                robot.forward(APPROACH_SPEED)
                
                # If we're very close, stop and capture
                if match_conf > 0.8:  # High confidence means we're close
                    robot.stop()
                    status_widget.value = f"Reached {target}!"
                    time.sleep(0.5)  # Wait for camera to stabilize
                    break
            elif x_offset > 0:
                # Object is to the right, turn right
                robot.right(APPROACH_SPEED * 0.8)
                time.sleep(0.1)
                robot.stop()
            else:
                # Object is to the left, turn left
                robot.left(APPROACH_SPEED * 0.8)
                time.sleep(0.1)
                robot.stop()
                
            # Move forward a bit
            robot.forward(APPROACH_SPEED)
            time.sleep(0.3)
            robot.stop()
        else:
            # If we lost the object, do a small scan to find it
            robot.left(SCAN_SPEED * 0.5)
            time.sleep(0.2)
            robot.stop()
    
    # Stop the robot
    robot.stop()
    status_widget.value = f"Taking picture of {target}"
    
    # Capture image
    capture_image(target)

# Turn 180°
def turn_180():
    status_widget.value = "Turning 180°"
    robot.left(SCAN_SPEED)
    time.sleep(3.0)  # Adjust time for smoother 180° turn
    robot.stop()

# Save image capture
def capture_image(obj_name):
    # Wait for camera to stabilize
    time.sleep(0.5)
    
    frame = camera.value
    if frame is None:
        print("Error: Could not capture frame")
        return
        
    # Save the raw image
    timestamp = int(time.time())
    path = os.path.join(SAVE_PATH, f"{obj_name}_{timestamp}.jpg")
    
    # Convert BGR to RGB for PIL
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = PILImage.fromarray(frame_rgb)
    img.save(path)
    
    print(f"Saved image at {path}")
    status_widget.value = f"Saved image of {obj_name}"

# Visit all objects in priority
def perform_patrol():
    log = []
    total_energy = 0
    total_time = 0

    for obj in PRIORITY_ORDER:
        t0 = time.time()
        status_widget.value = f"Looking for {obj}"
        
        # Scan until we find the current object
        found = False
        for _ in range(12):  # Try up to 12 scan segments
            robot.left(SCAN_SPEED)
            time.sleep(SCAN_STEP_TIME)
            robot.stop()
            
            obj_found, confidence = identify_object()
            if obj_found == obj:
                found = True
                break
        
        if found:
            go_to_object(obj)
            turn_180()  # After reaching object, turn 180°
        else:
            status_widget.value = f"Could not find {obj}"
            
        t1 = time.time()
        duration = t1 - t0
        energy = duration * 2  # Simplistic energy estimate
        total_time += duration
        total_energy += energy
        log.append((obj, round(duration, 2), round(energy, 2)))

    # Final stop
    robot.stop()

    # Print log
    print("\n--- PATROL LOG ---")
    for entry in log:
        print(f"Visited {entry[0]} | Time: {entry[1]}s | Energy: {entry[2]}")
    print(f"\nTotal Time: {round(total_time, 2)}s | Total Energy: {round(total_energy, 2)}")
    status_widget.value = "Patrol complete!"

# --- EXECUTION ---
if __name__ == "__main__":
    try:
        # Load reference images first
        load_reference_images()
        
        print("Starting patrol...")
        status_widget.value = "Starting initial scan..."
        
        # Scan for objects
        detected = scan_objects()
        
        # Check if all target objects were found
        missing = set(PRIORITY_ORDER) - detected
        if not missing:
            status_widget.value = "All objects found. Starting patrol."
            perform_patrol()
        else:
            status_widget.value = f"Missing objects: {', '.join(missing)}. Aborting."
            print("Not all target objects found. Aborting mission.")
            
    except KeyboardInterrupt:
        print("Mission aborted by user")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        # Always stop the robot and camera when done
        robot.stop()
        camera.stop()
        status_widget.value = "Mission complete."

SyntaxError: name 'PRIORITY_ORDER' is used prior to global declaration (<ipython-input-7-0a66a0ddaae7>, line 450)

In [ ]:
import time
import numpy as np
from jetbot import Robot
from jetbot import Camera
from PIL import Image as PILImage
import torch
import torch.nn as nn
from torchvision.models import mobilenet_v2
import torchvision.transforms as transforms
import os
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
import cv2

# --- CONFIGURABLE PARAMETERS ---
OBJECT_IMAGES_PATH = "dataset/"  # Folder containing reference object images
DETECTION_THRESHOLD = 0.6        # Minimum similarity threshold for object detection
APPROACH_SPEED = 0.2             # Speed for approaching objects
TURN_SPEED = 0.15                # Speed for turning when object not centered
CENTER_THRESHOLD = 30            # Pixels from center to consider object centered

# --- SETUP ---
robot = Robot()
camera = Camera.instance(width=224, height=224)

# Create widgets for visualization
image_widget = widgets.Image(format='jpeg', width=224, height=224)
status_widget = widgets.Text(value='Initializing...', description='Status:')
display(image_widget, status_widget)

# Define the function to convert BGR to JPEG for display
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])

# Link camera to image widget
camera_link = traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

# Load MobileNetV2 for feature extraction
model = mobilenet_v2(pretrained=True)
# Remove the classifier to get features instead of classification
feature_extractor = nn.Sequential(*list(model.children())[:-1])
feature_extractor.eval()

# Image preprocessing pipeline
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Object name to feature vector dictionary
object_features = {}

# Load object reference images and extract features
def load_reference_images():
    status_widget.value = "Loading reference images..."
    
    # Check if directory exists
    if not os.path.exists(OBJECT_IMAGES_PATH):
        status_widget.value = f"Error: directory {OBJECT_IMAGES_PATH} not found"
        return False
    
    # Get all jpg files in the directory
    object_files = [f for f in os.listdir(OBJECT_IMAGES_PATH) if f.endswith('.jpg')]
    
    if not object_files:
        status_widget.value = "Error: No jpg files found in dataset directory"
        return False
    
    for file in object_files:
        # Get object name from filename (remove .jpg extension)
        name = file.split('.')[0]
        img_path = os.path.join(OBJECT_IMAGES_PATH, file)
        
        try:
            image = PILImage.open(img_path).convert('RGB')
            image_tensor = transform(image).unsqueeze(0)
            
            # Extract features
            with torch.no_grad():
                features = feature_extractor(image_tensor)
                features = features.view(features.size(0), -1)  # Flatten
                object_features[name] = features
                
            print(f"Loaded reference image for {name}")
        except Exception as e:
            print(f"Error loading {name}: {e}")
    
    status_widget.value = f"Loaded {len(object_features)} reference objects"
    return len(object_features) > 0

# Cosine similarity for feature comparison
def cosine_similarity(t1, t2):
    t1_norm = torch.nn.functional.normalize(t1, p=2, dim=1)
    t2_norm = torch.nn.functional.normalize(t2, p=2, dim=1)
    return torch.mm(t1_norm, t2_norm.transpose(0, 1)).item()

# Capture current frame and compare with reference objects
def identify_object():
    frame = camera.value
    if frame is None:
        return None, 0.0
        
    # Convert BGR to RGB (OpenCV uses BGR, PIL expects RGB)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    img = PILImage.fromarray(frame_rgb)
    img_tensor = transform(img).unsqueeze(0)
    
    # Extract features
    with torch.no_grad():
        features = feature_extractor(img_tensor)
        features = features.view(features.size(0), -1)  # Flatten
    
    # Calculate similarity with each reference object
    similarities = {}
    for name, ref_features in object_features.items():
        sim = cosine_similarity(features, ref_features)
        similarities[name] = sim
    
    # Get the most similar object above threshold
    if similarities:
        best_match = max(similarities.items(), key=lambda x: x[1])
        if best_match[1] > DETECTION_THRESHOLD:
            return best_match
    
    return None, 0.0

# Detect object in frame and get position information
def detect_object_position(frame, object_name):
    h, w = frame.shape[:2]
    center_x, center_y = w // 2, h // 2
    
    try:
        # Load the reference image
        ref_path = os.path.join(OBJECT_IMAGES_PATH, f"{object_name}.jpg")
        template = cv2.imread(ref_path)
        template = cv2.resize(template, (w // 4, h // 4))  # Resize template
        
        # Convert both images to grayscale
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
        
        # Perform template matching
        result = cv2.matchTemplate(frame_gray, template_gray, cv2.TM_CCOEFF_NORMED)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        
        # Calculate center of matched region
        th, tw = template_gray.shape
        top_left = max_loc
        obj_center_x = top_left[0] + tw // 2
        obj_center_y = top_left[1] + th // 2
        
        # Draw rectangle and info on visualization
        vis_frame = frame.copy()
        cv2.rectangle(vis_frame, top_left, (top_left[0] + tw, top_left[1] + th), (0, 255, 0), 2)
        cv2.circle(vis_frame, (obj_center_x, obj_center_y), 5, (0, 0, 255), -1)
        cv2.line(vis_frame, (center_x, center_y), (obj_center_x, obj_center_y), (255, 0, 0), 2)
        cv2.putText(vis_frame, f"{object_name}: {max_val:.2f}", 
                   (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # Calculate offset from center (pixels)
        x_offset = obj_center_x - center_x
        
        # Update display
        image_widget.value = bgr8_to_jpeg(vis_frame)
        
        return (x_offset, max_val)
    except Exception as e:
        print(f"Error in position detection: {e}")
        return (0, 0)

# Main function to detect and approach objects
def detect_and_approach():
    try:
        # Load reference images first
        if not load_reference_images():
            print("Failed to load reference images. Exiting.")
            return
        
        status_widget.value = "Starting object detection..."
        
        while True:
            # Identify any object in the camera view
            object_name, confidence = identify_object()
            
            if object_name:
                status_widget.value = f"Detected {object_name} ({confidence:.2f})"
                
                # Get the current frame
                frame = camera.value
                if frame is None:
                    continue
                
                # Get object position
                x_offset, match_conf = detect_object_position(frame, object_name)
                
                # Decide how to move based on object position
                if abs(x_offset) < CENTER_THRESHOLD:
                    # Object is centered, move forward
                    status_widget.value = f"Moving toward {object_name}"
                    robot.forward(APPROACH_SPEED)
                    time.sleep(0.5)
                elif x_offset > 0:
                    # Object is to the right, turn right
                    status_widget.value = f"Turning right to center {object_name}"
                    robot.right(TURN_SPEED)
                    time.sleep(0.2)
                else:
                    # Object is to the left, turn left
                    status_widget.value = f"Turning left to center {object_name}"
                    robot.left(TURN_SPEED)
                    time.sleep(0.2)
                
                # Pause briefly
                robot.stop()
                time.sleep(0.1)
            else:
                # No object detected, slowly rotate to scan
                status_widget.value = "No objects detected, scanning..."
                robot.left(TURN_SPEED * 0.8)
                time.sleep(0.3)
                robot.stop()
                time.sleep(0.1)
    
    except KeyboardInterrupt:
        status_widget.value = "Stopped by user"
    except Exception as e:
        status_widget.value = f"Error: {e}"
    finally:
        # Always stop the robot and camera when done
        robot.stop()
        camera.stop()

# --- EXECUTION ---
if __name__ == "__main__":
    detect_and_approach()

Image(value=b'', format='jpeg', height='224', width='224')

Text(value='Initializing...', description='Status:')

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100.0%


Loaded reference image for case
Loaded reference image for bottle


In [12]:
import torch
import torchvision
from torchvision import transforms
import cv2
import numpy as np
from jetbot import Camera, Robot
from IPython.display import display, clear_output
import PIL.Image

# Load pre-trained object detection model (Faster R-CNN)

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
model.eval().cuda()

# COCO class labels
COCO_INSTANCE_CATEGORY_NAMES = [
    '_background_', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus',
    'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign',
    'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
    'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag',
    'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite',
    'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket',
    'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana',
    'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
    'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table',
    'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
    'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock',
    'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'
]

camera = Camera.instance(width=320, height=240)
robot = Robot()


transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()
])

def detect_and_act():
    try:
        while True:
            frame = camera.value
            img = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
            input_tensor = transform(img).unsqueeze(0).cuda()

            with torch.no_grad():
                outputs = model(input_tensor)[0]

            labels = outputs['labels'].cpu().numpy()
            scores = outputs['scores'].cpu().numpy()
            boxes = outputs['boxes'].cpu().numpy()

            # Default action
            action = "forward"

            for label, score, box in zip(labels, scores, boxes):
                if score < 0.5:
                    continue

                class_name = COCO_INSTANCE_CATEGORY_NAMES[label]
                x1, y1, x2, y2 = box.astype(int)

                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)

                # Example behavior logic
                if class_name == 'person':
                    action = "stop"
                elif class_name == 'knife':
                    action = "right"

            # Display action on screen
            cv2.putText(img, f'Action: {action}', (10, 230), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

            # Control robot movement
            if action == "stop":
                robot.stop()
            elif action == "right":
                robot.right(0.3)
            else:
                robot.forward(0.2)

            # Display image
            clear_output(wait=True)
            display(PIL.Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)))

    except KeyboardInterrupt:
        robot.stop()
        camera.stop()
        print("Stopped")

detect_and_act()

IndexError: list index out of range

In [11]:
camera.stop()

In [12]:
robot.stop()

In [41]:
for i in os.listdir('logs'):
    try:
        os.remove('logs/'+i)
    except IsADirectoryError:
        print()

In [18]:
for _ in range(36):  # 10° steps for full rotation
    robot.left(0.2)
    time.sleep(0.1)
    robot.stop()

In [43]:
import time
import torch
import torchvision.transforms as transforms
from torchvision.models import mobilenet_v2
from jetbot import Robot, Camera
from PIL import Image
import numpy as np
import json
import urllib.request

# Setup
robot = Robot()
camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)
display(image)

# Load ImageNet labels
url = 'https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt'
labels = urllib.request.urlopen(url).read().decode('utf-8').splitlines()


# Load pretrained Faster R-CNN
model = fasterrcnn_resnet50_fpn(pretrained=True).eval().cuda()

# Load pretrained model
model = mobilenet_v2(pretrained=True)
model.eval()

# Image transform
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Loop: check camera frame and act
try:
    print("Starting object monitor...")
    while True:
        frame = camera.value
        input_tensor = transform(frame).unsqueeze(0)

        with torch.no_grad():
            output = model(input_tensor)
            _, predicted = torch.max(output, 1)
            label = labels[predicted.item()]
            print(f"Detected: {label}")

            if label.lower() in ["bottle","switch","nipple","candle","oxygen mask"]:
                print("Bottle detected! Moving forward.")
                robot.forward(0.3)
            else:
                robot.stop()
        
        time.sleep(0.5)

except KeyboardInterrupt:
    print("Stopping.")
    robot.stop()
    camera.stop()


Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

Starting object monitor...
Detected: shower cap
Detected: plastic bag
Detected: piggy bank
Detected: Band Aid
Detected: toilet seat
Detected: oxygen mask
Bottle detected! Moving forward.
Detected: nipple
Bottle detected! Moving forward.
Stopping.


In [1]:
import time
import torch
import torch.nn.functional as F
from jetbot import Robot, Camera
from PIL import Image
import torchvision.transforms as T
from torchvision.models.detection import fasterrcnn_resnet50_fpn
import traitlets, ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg
from IPython.display import display

# ——— CONFIG ———
FORWARD_SPEED = 0.2
LOOP_DELAY    = 0.2   # seconds between frames
DETECT_THRESHOLD = 0.5

# COCO class IDs we care about
BOTTLE_CLASS    = 44
STOP_CLASSES    = {1, 27, 33}   # person=1, backpack=27, suitcase=33


# Load pretrained Faster R-CNN
model = fasterrcnn_resnet50_fpn(pretrained=True).eval().cuda()

# ——— SETUP ———
robot  = Robot()
camera = Camera.instance(width=640, height=480)

# Show camera in notebook
image_w = widgets.Image(format='jpeg', width=640, height=480)
traitlets.dlink((camera, 'value'), (image_w, 'value'),
                transform=bgr8_to_jpeg)
display(image_w)

# Preprocessing
transform = T.Compose([
    T.ToTensor(),      # scales to [0,1] and converts HWC→CHW
])

print("Starting object-detection loop (Ctrl+C to stop)…")
try:
    while True:
        frame = camera.value  # H×W×3 BGR uint8
        img   = frame[:, :, ::-1]  # BGR→RGB
        tensor = transform(img).cuda()

        # forward pass
        with torch.no_grad():
            outputs = model([tensor])[0]

        # gather detections above threshold
        labels = outputs['labels'].cpu().numpy()
        scores = outputs['scores'].cpu().numpy()

        # find highest-confidence bottle & stop-class detections
        bottle_detected = any((labels[i]==BOTTLE_CLASS and scores[i]>=DETECT_THRESHOLD)
                              for i in range(len(labels)))
        stop_detected   = any((labels[i] in STOP_CLASSES and scores[i]>=DETECT_THRESHOLD)
                              for i in range(len(labels)))

        # act
        if bottle_detected and not stop_detected:
            robot.forward(FORWARD_SPEED)
            print("→ Bottle only: MOVING forward")
        else:
            robot.stop()
            if stop_detected:
                print("■ Stop-class detected → STOP")
            elif not bottle_detected:
                print("■ No bottle detected → STOP")
            else:
                print("■ Bottle + stop-class overlap → STOP")

        time.sleep(LOOP_DELAY)

except KeyboardInterrupt:
    robot.stop()
    camera.stop()
    print("Stopped.")  


Error loading module `ublox_gps`: No module named 'serial'


Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100.0%


RuntimeError: Could not initialize camera.  Please see error trace.

In [48]:
# Real-time physics metrics for JetBot motion
import time
import math
from jetbot import Robot

# Constants for Waveshare JetBot
WEIGHT_KG = 0.8
BATTERY_VOLTAGE = 7.4  # volts
BATTERY_CAPACITY_MAH = 7800
BATTERY_CAPACITY_WH = (BATTERY_VOLTAGE * BATTERY_CAPACITY_MAH) / 1000  # watt-hours
BATTERY_CAPACITY_J = BATTERY_CAPACITY_WH * 3600  # convert to joules

robot = Robot()

# Initialize logging variables
start_time = time.time()
robot.forward(0.3)  # Move forward at 30% speed
print("Starting JetBot motion...")

# Real-time tracking loop
while True:
    current_time = time.time()
    elapsed_time = current_time - start_time
    if elapsed_time >= 5.0:  # Move for 5 seconds
        break

    # Simulated distance assuming speed 0.25 m/s
    speed = 0.25  # m/s
    distance = speed * elapsed_time  # m

    # Basic physics metrics
    acceleration = speed / elapsed_time  # m/s^2
    momentum = WEIGHT_KG * speed  # kg.m/s
    kinetic_energy = 0.5 * WEIGHT_KG * speed ** 2  # J
    work_done = WEIGHT_KG * acceleration * distance  # J
    power_output = work_done / elapsed_time if elapsed_time > 0 else 0  # W

    # Display metrics in real-time
    print("\n--- Real-Time JetBot Metrics ---")
    print(f"Time Elapsed: {elapsed_time:.2f} s")
    print(f"Estimated Distance: {distance:.2f} m")
    print(f"Speed: {speed:.2f} m/s")
    print(f"Momentum: {momentum:.2f} kg·m/s")
    print(f"Acceleration: {acceleration:.2f} m/s²")
    print(f"Work Done: {work_done:.2f} J")
    print(f"Kinetic Energy: {kinetic_energy:.2f} J")
    print(f"Power Output: {power_output:.2f} W")

    time.sleep(1)

# Stop JetBot
robot.stop()
print("\nJetBot stopped.")

Starting JetBot motion...

--- Real-Time JetBot Metrics ---
Time Elapsed: 0.03 s
Estimated Distance: 0.01 m
Speed: 0.25 m/s
Momentum: 0.20 kg·m/s
Acceleration: 8.64 m/s²
Work Done: 0.05 J
Kinetic Energy: 0.03 J
Power Output: 1.73 W

--- Real-Time JetBot Metrics ---
Time Elapsed: 1.04 s
Estimated Distance: 0.26 m
Speed: 0.25 m/s
Momentum: 0.20 kg·m/s
Acceleration: 0.24 m/s²
Work Done: 0.05 J
Kinetic Energy: 0.03 J
Power Output: 0.05 W

--- Real-Time JetBot Metrics ---
Time Elapsed: 2.04 s
Estimated Distance: 0.51 m
Speed: 0.25 m/s
Momentum: 0.20 kg·m/s
Acceleration: 0.12 m/s²
Work Done: 0.05 J
Kinetic Energy: 0.03 J
Power Output: 0.02 W

--- Real-Time JetBot Metrics ---
Time Elapsed: 3.04 s
Estimated Distance: 0.76 m
Speed: 0.25 m/s
Momentum: 0.20 kg·m/s
Acceleration: 0.08 m/s²
Work Done: 0.05 J
Kinetic Energy: 0.03 J
Power Output: 0.02 W

--- Real-Time JetBot Metrics ---
Time Elapsed: 4.04 s
Estimated Distance: 1.01 m
Speed: 0.25 m/s
Momentum: 0.20 kg·m/s
Acceleration: 0.06 m/s²
Work D